In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
# import keras_metrics as km
# from keras import regularizers, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
import math

In [ ]:
data_dir = "/content/drive/MyDrive/video_data"
img_height , img_width = 64, 64
seq_len = 16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
classes = ["fall", "fall_down", "grab", "swallow", "walk","sit","climb"]

In [ ]:
def frames_extraction(video_path):
    frames_list = []
     
    vidcap = cv2.VideoCapture(video_path)
    total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

    frames_step = total_frames//seq_len
 
    for j in range(seq_len):
        #here, we set the parameter 1 which is the frame number to the frame (i*frames_step)
        vidcap.set(1,j*frames_step)
        success,image = vidcap.read()  
        #save your image
        if success:
          image = cv2.resize(image, (img_height, img_width))
          frames_list.append(image)
        else:
          print("Defected frame {}".format(video_path))
          break

    return frames_list

In [ ]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [ ]:
X, Y = create_data(data_dir)
X = X/255
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)

walk
Defected frame /content/drive/MyDrive/video_data/walk/135.mov
Defected frame /content/drive/MyDrive/video_data/walk/143.mov
Defected frame /content/drive/MyDrive/video_data/walk/101.mov
fall_down
Defected frame /content/drive/MyDrive/video_data/fall_down/145.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/143.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/172.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/21.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/26.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/46.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/52.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/57.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/81.mov
swallow
Defected frame /content/drive/MyDrive/video_data/swallow/47.mov
Defected frame /content/drive/MyDrive/video_data/swallow/70.mov
Defected frame /content/drive/MyDrive/video_data/swallow/79_crop (

In [ ]:
model = Sequential()

model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3),strides = (2,2), return_sequences = True, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3),strides = (2,2), return_sequences = True, data_format = "channels_last"))
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3),strides = (2,2), return_sequences = True, data_format = "channels_last"))
model.add(ConvLSTM2D(filters = 128, kernel_size = (3, 3),strides = (2,2), return_sequences = False, data_format = "channels_last"))


model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(7, activation = "softmax"))
 
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_12 (ConvLSTM2D) (None, 16, 31, 31, 32)    40448     
_________________________________________________________________
conv_lst_m2d_13 (ConvLSTM2D) (None, 16, 15, 15, 32)    73856     
_________________________________________________________________
conv_lst_m2d_14 (ConvLSTM2D) (None, 16, 7, 7, 64)      221440    
_________________________________________________________________
conv_lst_m2d_15 (ConvLSTM2D) (None, 3, 3, 128)         885248    
_________________________________________________________________
flatten_4 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               590336    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)              

In [ ]:
def schedular(epoch):
  if epoch < 10:
    return 0.001
  else:
    return 0.001 * math.exp(0.1*(10-epoch))

In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
callbacks = tf.keras.callbacks.LearningRateScheduler(schedular)
 
history = model.fit(x = X_train, y = y_train, epochs=20, batch_size = 64, shuffle=True, validation_split=0.3, callbacks=callbacks)

import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val','accuracy','val_accuracy'])
plt.show()


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
17/17 [==============================] - 174s 10s/step - loss: 1.9246 - accuracy: 0.1626 - val_loss: 1.8906 - val_accuracy: 0.2179
Epoch 2/50
17/17 [==============================] - 162s 10s/step - loss: 1.8849 - accuracy: 0.2171 - val_loss: 1.8046 - val_accuracy: 0.2529
Epoch 3/50
17/17 [==============================] - 163s 10s/step - loss: 1.8042 - accuracy: 0.2726 - val_loss: 1.7200 - val_accuracy: 0.2918
Epoch 4/50
17/17 [==============================] - 163s 10s/step - loss: 1.6524 - accuracy: 0.3496 - val_loss: 1.6554 - val_accuracy: 0.3930
Epoch 5/50
17/17 [==============================] - 162s 10s/step - loss: 1.5195 - accuracy: 0.4080 - val_loss: 1.5858 - val_accuracy: 0.4202
Epoch 6/50
17/17 [==============================] - 162s 10s/step - loss: 1.3118 - accuracy: 0.5073 - val_loss: 1.5914 - val_accuracy: 0.4475
Epoch 7/50
17/17 [==============================] - 163s 10s/step - loss: 1.0159 - accuracy: 0.6212 - val_loss: 1.6384 - val_accuracy: 0.4397
Epoch 

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test) 
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)
 
print(classification_report(y_test, y_pred))

print('accuracy score : {}'.format(accuracy_score(y_pred, y_test)))
